# Installing dependences

In [ ]:
!pip install datasets
!pip install huggingface_hub
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

# Importing libraries

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random
from datasets import load_dataset
from huggingface_hub import login
dataset = load_dataset("Open-Orca/OpenOrca")

# Getting the dataset

In [ ]:
original_dataset = dataset['train']
sample_size = 10000  # Since resources were low I used random 10000 size
random_indices = random.sample(range(0, len(original_dataset)), sample_size)
sampled_dataset = original_dataset.select(random_indices)
print(sampled_dataset)

Dataset({
    features: ['id', 'system_prompt', 'question', 'response'],
    num_rows: 10000
})


# Removed instructions with less than 100 tokens in response.

In [ ]:
def is_long_enough(example):
    num_tokens = len(example['response'].split())
    return num_tokens >= 100

In [ ]:
filtered_dataset = sampled_dataset.filter(is_long_enough)


Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

# Data deduplication by doing grouping using cosine similarity (threshold>0.95)

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
responses = filtered_dataset['response']
embeddings = model.encode(responses)
cos_sim_matrix = cosine_similarity(embeddings, embeddings)
duplicates = (cos_sim_matrix > 0.95) & ~np.eye(cos_sim_matrix.shape[0], dtype=bool)
to_remove = set()
for i in range(duplicates.shape[0]):
    for j in range(i + 1, duplicates.shape[1]):
        if duplicates[i, j]:
            to_remove.add(j)
unique_indices = [i for i in range(len(responses)) if i not in to_remove]
unique_dataset = filtered_dataset.select(unique_indices)


In [ ]:
print(unique_dataset)

Dataset({
    features: ['id', 'system_prompt', 'question', 'response'],
    num_rows: 2878
})


# publishing this dataset on huggingface

In [ ]:
login()

In [ ]:
unique_dataset.push_to_hub("Jasteg19/Ocra_Sample_Dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Jasteg19/Ocra_Sample_Dataset/commit/80efaf8fbbb66109e69cf082c3395f9cccc9a2b4', commit_message='Upload dataset', commit_description='', oid='80efaf8fbbb66109e69cf082c3395f9cccc9a2b4', pr_url=None, pr_revision=None, pr_num=None)